## Exercises

### Data Acquisition

These exercises should go in a notebook or script named `wrangle`. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the
san antonio 311 call dataset.

1. Read the case, department, and source data into their own spark dataframes.

1. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json
      format, store these as `sources_csv` and `sources_json`
    - Inspect your folder structure. What do you notice?

9. Inspect the data in your dataframes. Are the data types appropriate? Write
   the code necessary to cast the values to the appropriate types.


---

1. How old is the latest (in terms of days past SLA) currently open issue? How
   long has the oldest (in terms of days since opened) currently opened issue
   been open?
1. How many Stray Animal cases are there?
1. How many service requests that are assigned to the Field Operations
   department (`dept_division`) are not classified as "Officer Standby" request
   type (`service_request_type`)?

1. Convert the `council_district` column to a string column.
1. Extract the year from the `case_closed_date` column.
1. Convert `num_days_late` from days to hours in new columns `num_hours_late`.

1. Join the case data with the source and department data.
1. Are there any cases that do not have a request source?

1. What are the top 10 service request types in terms of number of requests?
1. What are the top 10 service request types in terms of average days late?
1. Does number of days late depend on department?
1. How do number of days late depend on department and request type?

In [1]:
import pyspark
#creates the instance of spark and connects to the java virt machine, how we call the spark methods
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
import pandas as pd
import numpy as np
from pydataset import data
from pyspark.sql.functions import col, expr
# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import month, year, quarter